In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
sind_strs = [ 'S01', 'S02', 'S03', 'S04', 'S05', 'S07' ]

In [ ]:
import sys,os
p = '$HOME/osccode/data_proc'
p = os.path.expandvars(p)
print(p)
sys.path.append(p)

In [ ]:
%env OSCBAGDIS_DATAPROC_CODE=/home/demitau/osccode/data_proc

In [ ]:
#sind_strs = [ 'S01', 'S02', 'S03', 'S04', 'S05', 'S07' ]

import globvars as gv
from os.path import join as pjoin
from utils import loadROILabels, loadSurfAndGrids
# it is very important to get the right order
roi_labels, srcgroups, source_coords = loadROILabels()
vis_info_per_subj = loadSurfAndGrids(sind_strs,None)
cgMNI = vis_info_per_subj['S_default']

In [ ]:
import numpy as np
for sind_str in sind_strs:
    rncur = f'{sind_str}_on_move'
    sources_type = 'parcel_aal'
    src_file_grouping_ind = 10
    src_rec_info_fn = '{}_{}_grp{}_src_rec_info'.format(rncur,
                                                        sources_type,src_file_grouping_ind)
    d = '/home/demitau/ju_oscbagdis/data_duss/feats_wholectx'
    src_rec_info_fn_full = os.path.join(d, src_rec_info_fn + '.npz')
    rec_info = np.load(src_rec_info_fn_full, allow_pickle=True)


    b = 'new_src_order' in list( rec_info.keys() )
    print( f'{sind_str} {b}')

In [ ]:
mtime = os.stat(src_rec_info_fn_full).st_mtime
import datetime  as dtdt
str( dtdt.datetime.fromtimestamp(mtime) )

In [ ]:
#cd ..

In [ ]:
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj, RoiObj

In [ ]:
# TODO: make colorbar numbers corresond to data, make sure I export the right thing (now maybe I export something wrong)

In [ ]:
vals_per_source_list = []
clim = None

In [ ]:
#info['color_group_labels']
#info['impr_per_medcond_per_pgn']
#info.keys()

# plotting brain maps for merged across subj, within medcond

In [ ]:
intensities_all_valid = []
ttl = 'brain_map_area_strength'
modes = ['only'];        #ttl = 'brain_map_area_strength'
#modes = ['LFPand_only']; #ttl = 'brain_map_LFP_and_area_strength'
for mode in  modes:  # exclude
    for medcond in ['on', 'off']:
        fname_full = pjoin(gv.data_dir,subdir,f'EXPORT_{ttl}_medcond={medcond}_mode={mode}.npz')
        f = np.load(fname_full,allow_pickle=1)
        info = f['info'][()]

        print(mode,medcond)
        intensities = info['intensities']
        intensities_all_valid += [intensities[1:]]
        d = dict( zip(info['brain_area_labels'], intensities) )
        print( d['Cerebellum'] )
        
ddgood = np.array(intensities_all_valid)
clim = np.min(ddgood), np.max(ddgood)
print(clim)

####################################################################
import plots, utils
from plots import plot3DValPerParcel
fit_to_brain = 1
use_common_colorbar = 0
if not use_common_colorbar:
    clim = None

renders = []
infos = []

radius_project = 1.3    
subdir = 'joint_noskip'
subdir_fig = 'output'
plot_intremed = False

for mode in  modes:  # exclude
    for medcond in ['on', 'off']:
        fname_full = pjoin(gv.data_dir,subdir,f'EXPORT_{ttl}_medcond={medcond}_mode={mode}.npz')
        f = np.load(fname_full,allow_pickle=1)
        info = f['info'][()]
        infos += [info]
        
        d = dict( zip(info['brain_area_labels'], info['intensities']) )        
        #intensitites = np.array(info['intensities'][1:] ) #/100
        intensitites = np.array(info['intensities'] ) #/100

        vals_per_source = utils.dupValsWithinParcel(info['brain_area_labels'],info['srcgrp_new'], intensitites)
        vals_per_source_list += [vals_per_source.copy()]
#         bads = np.isnan(vals_per_source)
#         #vals_per_source[bads ] = np.min( vals_per_source[ ~bads ]  )
#         vals_per_source[ bads ] = 0
        print('vals_per_source  min', np.min( vals_per_source[~np.isnan(vals_per_source) ]) )

        vi = vis_info_per_subj[sind_str]
        vi['headsurfgrid_verts'] = info['coords']

        title = f'medcond={medcond.upper()}_mode={mode}_fit{fit_to_brain}'
        val_LFP = info['impr_per_medcond_per_pgn'][medcond].get('LFP', None)
        print(f'title={title}, CB intensity={d["Cerebellum"]}, LFP={val_LFP}, clim={clim}' )
        #continue
        figtitle = f'medcond={medcond.upper()}'
        #for fit_to_brain in [ 0,1]:
        #info['srcgrp_new'], info['color_group_labels'],
        sc = plot3DValPerParcel(vals_per_source, val_LFP, vis_info_per_subj[sind_str],
                           figtitle, show_supp=False, radius_project=radius_project, 
                           brain_translucent=False , bgcolor='black',
                          sources_visible = False, 
                               fit_to_brain= fit_to_brain,
                              clim = clim, views=['left', 'right', 'bottom'], 
                                ww = 500*4 + 200, hh = 600)

                
        figfname_full= pjoin(gv.code_dir, subdir_fig, f'{title}_visbrain_rad{radius_project}.png')
        render_result = plots.saveRenderVisBrainScene(sc,figfname_full, render_only = not plot_intremed)
        
        #sc,render_result = r
        renders += [render_result]
        
renders_joined = np.vstack(renders)
fig2_fname_full= pjoin(gv.code_dir, subdir_fig, f'{ttl}_{mode}_joined_visbrain_rad{radius_project}_commoncb{use_common_colorbar}.png')
plots.array2png(renders_joined,fig2_fname_full)
        

# plotting brain maps for merged within medcond

In [ ]:
import globvars

In [ ]:
vals_per_source_list = []
clim = None
#subdir = 'per_subj_per_medcond_best_LFP'
subdir = 'per_subj_per_medcond_best_LFP_wholectx'

In [ ]:
countinfo

In [ ]:
import pandas as pd
beh_dur_df = pd.DataFrame({'rawname': [], 'prefix':[], 'grp':[], 'it':[],
                           'subj':[], 'medcond':[], 'mvt_side':[],
                            'trem':[], 'rest':[], 'hold':[], 'move':[],
                           'total':[], 'bacc_pct':[]  })

intnameconv=dict(trem='trem',notrem='rest',hold='hold',move='move')

not_num_cols = 4 +3 

fn_full  = pjoin(gv.data_dir,subdir,'beh_states_durations.npz')
countinfo = np.load(fn_full, allow_pickle=1)['arr_0'][()]

from globvars import gp
#countinfo
countinfo3 = {}
countinfo4 = {}
baccpcts = []
for k,item in countinfo.items():
    cts = {}
    l1,l2 = [],[]
    counts = item['counts']
    if 'move_L' in counts:
        movelet = 'left'
    else:
        'move_R' in counts
        movelet = 'right'
    newrow = [*k] + k[0].split('_') + [movelet] + [0]*(4+2)
    # we want sorted
    totlen = 0
    for kk in gp.int_types_basic_sided:
        if kk not in counts:
            continue
        else:
            ct = counts[kk]
        ctsec = ct /32
        kk_noside = kk[:-2]
        #colind = not_num_cols + gp.int_types_basic.index(kk_noside)
        colind =  list(beh_dur_df.columns).index(
            intnameconv[kk_noside])
    #for kk,ct in counts.items():
        scur = f'{ctsec:.0f}s'
        cts[kk.split('_')[0]] = scur
        if kk.startswith('notrem'):
            letter = 'Q'
        elif kk.startswith('move'):
            letter = 'G'
        else:
            letter = kk[0].upper()
        
        totlen += ctsec
        newrow[colind] = ctsec
            
        l1 += [letter]
        l2 += [scur]    
    newrow[list(beh_dur_df.columns).index('total')] = totlen
    baccpct = item['bacc'] * 100
    baccpcts += [baccpct]
    cts['bacc'] = f'{int(baccpct)}%'
    countinfo3[k[0]] = cts
    countinfo4[k[0]] = f"{','.join(l1)}={','.join(l2)}"
    
    newrow[-1] = baccpct
    
    beh_dur_df.loc[len(beh_dur_df.index)] = newrow
    
    
bacc_me, bacc_std = np.nan, np.nan    
    
beh_dur_df.loc[len(beh_dur_df.index)] = newrow    
    
display(countinfo3)
display(countinfo4)



countinfo2 = {}
for k,item in countinfo.items():
    countinfo2[k[0]] = item['infostr']
countinfo2

In [ ]:
# for mc in ['off', 'on']:
#     print(mc)
#     df_cur_mc = beh_dur_df[beh_dur_df['medcond'] == mc]
#     display(df_cur_mc)

In [ ]:
# means
for mc in ['off', 'on']:
    print(mc)
    df_cur_mc = beh_dur_df[beh_dur_df['medcond'] == mc]
    me = df_cur_mc.mean(axis=0, numeric_only=True)
    std = df_cur_mc.std(axis=0, numeric_only=True)
    display(me)
    
    newrow = ['mean'] + [0]*(not_num_cols-1) + list(me)
    df_cur_mc.loc[max(df_cur_mc.index)]= newrow
    
    df_cur_mc = df_cur_mc.sort_index().reset_index(drop=True)
    
    newrow2 = ['std'] + [0]*(not_num_cols-1) + list(std)
    df_cur_mc.loc[max(df_cur_mc.index) + 3]= newrow2
    
    df_cur_mc = df_cur_mc.sort_index().reset_index(drop=True)
    
    display(df_cur_mc)

In [ ]:
#for index, row in df.iterrows():
#    print(row['c1'], row['c2'])

In [ ]:
#subj2marker

In [ ]:
import matplotlib.pyplot as plt
mc2col = {'off':'red', 'on':'green'}
itnames_full={'trem':'tremor','rest':'rest', 
              'hold':'hold', 'move':'move'}
fig,axs = plt.subplots(2,2, figsize=(7,7))
axs = axs.reshape(4)
ind = 0
for it in itnames_full:    
    for mc in ['off', 'on']:
        ax = axs[ind]
        print(mc)
        df_cur_mc = beh_dur_df[beh_dur_df['medcond'] == mc]

        trem_durs = df_cur_mc[it]
        baccs = df_cur_mc['bacc_pct']

        ax.scatter(trem_durs,baccs,c=mc2col[mc],label=mc.upper() )
    ax.legend()
    ax.set_xlabel( f'{itnames_full[it]} duration [s]'   )
    ax.set_ylabel( 'balanced accuracy [%]' )
    ax.set_ylim(40,100)
    ind += 1
plt.tight_layout()
plt.savefig('output/dur2bacc_scatters.pdf')

In [ ]:
list(beh_dur_df.loc[0])[:4] + list(beh_dur_df.mean(axis=0, numeric_only=True))

In [ ]:
beh_dur_df.mean(axis=0, numeric_only=True)

In [ ]:
list(beh_dur_df.mean(axis=0, numeric_only=True))

In [ ]:
display(countinfo3)

In [ ]:
#gp.parcel_groupings_post_sided

In [ ]:
impr

In [ ]:
#info['impr_per_medcond_per_pgn'][medcond]  # has both sides and does not have LFP
info2['impr_per_medcond_per_pgn'][medcond]  # has both sides and also does not have LFP

In [ ]:
# We need to assign to each source a parcel group (not parcel like it was before) it belongs to
#roi_labels



## Preparing clim

In [ ]:
import traceback
import plots, utils
from plots import plot3DValPerParcel

info0 = None
intensities_all_valid = []
infos_all = {}
#modes = ['only'];        #ttl = 'brain_map_area_strength'
#modes = ['LFPand_only']; #ttl = 'brain_map_LFP_and_area_strength'
mode = 'LFPand_only'
for sind_str in sind_strs:
    infos_all[sind_str] = {}
    #for mode in  modes:  # exclude
    infos_all[sind_str][mode] = {}
    for medcond in ['on', 'off']:     
        if medcond == 'on' and sind_str == 'S03':
            continue
        infos_all[sind_str][mode][medcond] = {}
        ttl = f'brain_map_area_strength_{sind_str}_{medcond}'
        mode = 'LFPand_only'
        fname_full = pjoin(gv.data_dir,subdir, f'EXPORT_{ttl}_medcond={medcond}_mode={mode}.npz')
        f = np.load(fname_full,allow_pickle=1)
        info_rel_LFP = f['info'][()]  

        fname_full2 = pjoin(gv.data_dir,subdir, f'EXPORT_{ttl}_medcond={medcond}_mode=only.npz')
        f2 = np.load(fname_full2,allow_pickle=1)
        info2 = f2['info'][()]

        infos_all[sind_str][mode][medcond]['info_pgn_rel_LFP'] = info_rel_LFP  #LFPand_only
        infos_all[sind_str][mode][medcond]['info_pgn_abs'] = info2 

        # consistency check just to calm nerves
        for k,v in info_rel_LFP['impr_per_medcond_per_pgn']['off'].items():
            if k.startswith('base_'):
                continue
            assert v == info_rel_LFP['intensities'][ info_rel_LFP['brain_area_labels'].index(k) ]

        print(mode,medcond)
        intensities = info_rel_LFP['intensities']
        intensities_all_valid += [intensities[1:]]
        d = dict( zip(info['brain_area_labels'], intensities) )
        #print( d['Cerebellum'] )

        if sind_str == 'S01':
            info0 = info
        

In [ ]:
ints_av = np.array( intensities_all_valid ).flatten()
plt.hist(ints_av);

In [ ]:
# mc = 'on'
# i1 = infos_all['S01']['LFPand_only'][mc]['info']['impr_per_medcond_per_pgn'][mc]
# i2 = infos_all['S01']['LFPand_only'][mc]['info_only']['impr_per_medcond_per_pgn'][mc]
# display(i1)
# display(i2)

In [ ]:
info['impr_per_medcond_per_pgn']

In [ ]:
intensitites

In [ ]:
###
from plots import plotBrainPerSubj
ddgood = np.array(intensities_all_valid)
ddgood = ddgood[~np.isnan(ddgood)]
clim = np.min(ddgood), np.max(ddgood)
print(clim)
save = 0

plotBrainPerSubj(sind_strs, vis_info_per_subj, source_coords, 
                 subdir, clim, countinfo4, fix_vis_info=True, verbose=2, save=save)

if save:        
    renders_joined = np.vstack(renders)
    fig2_fname_full= pjoin(gv.code_dir, subdir_fig, 
        f'{ttl}_{mode}_joined_visbrain_rad{radius_project}_commoncb{use_common_colorbar}.png')
    plots.array2png(renders_joined,fig2_fname_full)

    if not show_supp:
        fig2_fname_table_full= pjoin(gv.code_dir, subdir_fig, f'{ttl}_{mode}_joined_table_visbrain_rad{radius_project}_commoncb{use_common_colorbar}.png')
        plots.array2png(all_renders,fig2_fname_table_full)

    fname_zeros_full = pjoin(gv.code_dir,subdir_fig,'zeros.png')
    plots.array2png(render_result*0,  fname_zeros_full)

In [ ]:
gv.dir_fig

In [ ]:
for pik,piv in plotinfos.items():
    print( pik, piv.get('LFP_val_mode=only',np.nan) )

In [ ]:
plotinfo_cur['LFP_val_mode=only']

In [ ]:
#info2['impr_per_medcond_per_pgn']

## Gather final

In [ ]:
import matplotlib.pyplot as plt
plt.rc('axes', titlesize=18)
plt.rc('ytick', labelsize=18)

tpls = []
for rowi,sind_str in enumerate(sind_strs):
#for sind_str in ['S03']:
    for mode in  modes:  # exclude
        for mmci,medcond in enumerate( ['off', 'on'] ):        
            plotinfo_cur = plotinfos[(sind_str,medcond,mode)]
            p = plotinfo_cur.get('figfname_full',fname_zeros_full)
#             p = plotinfo_cur.get('render_result',None)
#             if p is None:
#                 p = np.zeros((100,100,4))
            LFP_val = plotinfo_cur.get('LFP_val_mode=only',np.nan)
            if 'figfname_full' not in plotinfo_cur:
                ttl = ''
            else:
                #ttl = f'{sind_str} {medcond.upper()}; LFP perf = {LFP_val:.1f}'
                ttl = plotinfo_cur['figtitle']
            tpl = (p, ttl)
            tpls += [tpl ]
fnames,titles = zip(*tpls)
            
from visbrain.gui import Figure

grid_dim = len(plotinfos) // 2, 2
#figsize = (grid_dim[0] * 500, grid_dim[1] * 500)
figsize = (grid_dim[1] * 5, grid_dim[0] * 4 - 7)

subspace = {'wspace': 0.1, 'left': 0., 'wspace':0.15, 'hspace':0.05}
f = Figure(fnames, titles=titles, figtitle='Performances per subject', 
           grid=grid_dim, ax_bgcolor='black',
           y=1., fig_bgcolor='black', figsize=figsize,
           text_color='white', subspace=subspace, autoresize=1, autocrop=1)

clim = np.round( np.array( clim ), 1 )
ticks = [clim[0], 0., clim[1] ]
#ticks = [clim[0], 0, np.round(clim[1],1) ]
f.shared_colorbar(clim=clim, cmap='plasma', fz_title=15,
                  under='olive', over='firebrick', position='right',
                  title='Performance increase, compared to LFP alone', 
                  fz_ticks=15, pltmargin=.05,
                  figmargin=.1, ticks=ticks, ycb=15)

figname = 'Brains_for_pub.png'
figname_full = pjoin(gv.code_dir,subdir_fig,figname)
f.save(figname_full, dpi=300);
#f.close()
#plt.close()

In [ ]:
sc.canvas.size

In [ ]:
renders_joined = np.vstack(renders)
sstr = ','.join(sind_strs)
fig2_fname_full= pjoin(gv.code_dir, subdir_fig, f'{sstr}_{mode}_joined_visbrain_rad{radius_project}_commoncb{use_common_colorbar}.png')
plots.array2png(renders_joined,fig2_fname_full)

In [ ]:
EXPORT_brain_map_LFP_and_area_strength_medcond=on_mode=only.npz
EXPORT_brain_map_LFP_and_area_strength_medcond=on_mode=LFPand_only.npz

In [ ]:
#infos

## Create duration table

In [ ]:

subjs = sorted(set([a.split('_')[0] for a in countinfo3.keys()] ) )
medconds = sorted(set([a.split('_')[1] for a in countinfo3.keys()] ) )
bstates = list( list(countinfo3.values() )[0].keys()  )  # fixed order

lines = []
line = 'subj,'
for mc in medconds:
    for bc in bstates:
        line += f'{mc.upper()}_{bc},'
lines += [line+ '\n']
for s in subjs:
    line = f'{s},'
    for mc in medconds:
        rn = f'{s}_{mc}'
        d = countinfo3.get(rn,None)        
        for bc in bstates:            
            if d is None:
                v = ''
            else:
                v = d[bc]#[:-1]
            line +=  f'{v},'
    lines += [line + '\n']

display(lines)
sd = pjoin(gv.dir_fig,subdir)
if not os.path.exists(sd):
    os.makedirs(sd)
durtable_fn = pjoin(sd,'duration_table.csv')
with open(durtable_fn,'w') as f:
    f.writelines(lines)
    

In [ ]:
info['brain_area_labels']

In [ ]:
vals_per_source = utils.dupValsWithinParcel(info['brain_area_labels'],info['srcgrp_new'], intensitites)
cb_inds = np.where( info['srcgrp_new'] == info['brain_area_labels'].index('Cerebellum') )[0]
cb_vals = vals_per_source[cb_inds]
cb_vals

In [ ]:
for vv in vals_per_source_list[-2:]:
    ar = vv[~np.isnan(vv)]
    a = np.min(  ar)
    aa = np.max(  ar)
    cb_vals = vv[cb_inds]
    print(a,aa, cb_vals, np.max(ar))

In [ ]:
renders_joined = np.vstack(renders)  # vertically,  hstack would be horizontally
fig2_fname_full= pjoin(gv.code_dir, subdir_fig, f'joined_visbrain_rad{radius_project}.png')
plots.array2png(renders_joined,fig2_fname_full)


In [ ]:
from visbrain.objects import SceneObj, BrainObj, SourceObj, RoiObj


tris = vi['headsurf_tris']
verts =  vi['headsurf_verts'] * 10 #- 1.

tc = True
b_obj = BrainObj(f'brainsurf', translucent=tc, vertices=verts,
                    faces=tris,  hemisphere='both')


roi_aal = RoiObj('aal')
idx_th = roi_aal.where_is('Thalamus (L)')
r = roi_aal.select_roi(select=idx_th)

mean_v = np.mean(roi_aal.vertices, axis=0)
xyz = mean_v[None,:]
radius_supp = 2
RADINFO = dict(radius_min=radius_supp, radius_max = radius_supp)
s_obj  = SourceObj ('mysrc', xyz, data=[1.], cmap='inferno', **RADINFO)

s_obj.project_sources(roi_aal, cmap='plasma', radius = 20, clim=(-1,1))
#, clim=(-1., 1.), vmin=-.5, vmax=.7, under='gray', over='red', radius = 5

sc = SceneObj(bgcolor='white', size=(1500, 500))
sc.add_to_subplot(b_obj, rotate='left', zoom=2)
sc.add_to_subplot(roi_aal, rotate='left')
sc.add_to_subplot(s_obj, rotate='left')


sc.preview()

In [ ]:
help(sc.preview)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(  roi_aal.vertices[:,0] )

In [ ]:
lvls = [7, 12, 20, 36, 64, 122]
lvl = lvls[5]; print(lvl)

In [ ]:
roi_dmn = RoiObj(f'mist_{lvl}')
lbls2 = roi_dmn.get_labels()

In [ ]:
lbls2.columns

In [ ]:
for l in list(lbls2[f'name_{lvl}']):
    if l.lower().find('thal') >= 0:
        print(l)

In [ ]:
#idx_th

In [ ]:
idx_th = roi_dmn.where_is('Thalamus ventral')
roi_dmn.select_roi(select=idx_th)
plt.hist(  roi_dmn.vertices[:,0] )

In [ ]:
list( lbls2[f'name_{lvl}'] )

In [ ]:
help(roi_aal.select_roi)

In [ ]:
lbls = roi_aal.get_labels()

In [ ]:
list( lbls.aal )

In [ ]:
roi_aal.where_is

In [ ]:
roi_to_find1('brodmann')                      # Switch to Brodmann
idx_ba6 = roi_to_find1.where_is('BA6')        # Find only BA6
print(ref_brod.loc[idx_ba6])

In [ ]:
import imageio
from skimage.io import imsave
import numpy as np
import imageio

In [ ]:
import cv2

In [ ]:
im = np.flip(sc.render(),axis=0)

#import scipy.misc
figname_full =  pjoin(gv.code_dir,'output','test.png')


imageio.imwrite(figname_full, im )

#imsave(figname_full, im )
#cv2.imwrite(figname_full, im )


In [ ]:
help(sc.render)

In [ ]:
from mpl_render import RenderingImShow
help(RenderingImShow)

In [ ]:
vals_per_source

In [ ]:
info

In [ ]:
#srcgroups % 2
srcgroups

In [ ]:
roi_labels

In [ ]:
((583, 3), (583,))

In [ ]:
#vi['headsurfgrid_verts'].shape


Brain object (BrainObj) : complete tutorial
===========================================

This example illustrate the main functionalities and inputs of the brain
object i.e :

  * Use included MNI brain template
  * Select the hemisphere ('both', 'left', 'right')
  * Use a translucent or opaque brain
  * Project source's activity on the surface of the brain
  * Parcellize the brain and send data to selected parcellates
  * Add fMRI activation and MEG inverse solution

Data for fMRI activations and MEG inverse solutoin comes from the PySurfer
software (https://github.com/nipy/PySurfer/). Parcellation file comes from
MNE-Python (https://github.com/mne-tools/mne-python).



In [ ]:
import numpy as np

from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
from visbrain.io import download_file, read_stc

Scene creation
##############################################################################
 The SceneObj is Matplotlib subplot like in which, you can add visbrain's
 objects. We first create the scene with a black background, a fixed size



In [ ]:
# Scene creation
sc = SceneObj(bgcolor='black', size=(1400, 1000))
# Colorbar default arguments. See `visbrain.objects.ColorbarObj`
CBAR_STATE = dict(cbtxtsz=12, txtsz=10., width=.1, cbtxtsh=3.,
                  rect=(-.3, -2., 1., 4.))
KW = dict(title_size=14., zoom=1.2)

<div class="alert alert-info"><h4>Note</h4><p>The BrainObj can interact with sources (SourceObj). For example, if the
    source object represent intracranial data (e.g iEEG) those sources can
    be projected on the surface of the brain. This is an important feature
    because intracranial implantations is usually subject dependant and the
    projection is a good way to plot results across subjects. To illustrate
    this feature, we provide a set of intracranial MNI coordinates.</p></div>



In [ ]:
# Download iEEG coordinates and define some random data
mat = np.load(download_file('xyz_sample.npz', astype='example_data'))
xyz, subjects = mat['xyz'], mat['subjects']
data = np.random.rand(xyz.shape[0])

In [ ]:
xyz.shape, data.shape

Basic brain using MNI template
##############################################################################
 By default, Visbrain include several MNI brain templates (B1, B3, B3,
 inflated, white and shere).



In [ ]:
# Translucent inflated BrainObj with both hemispheres displayed
b_obj_fs = BrainObj('inflated', translucent=True, hemisphere='both')
# Add the brain to the scene. Note that `row_span` means that the plot will
# occupy two rows (row 0 and 1)
sc.add_to_subplot(b_obj_fs, row=0, col=0, row_span=2,
                  title='Translucent inflated brain template', **KW)

Select the left or the right hemisphere
##############################################################################
 You can use the `hemisphere` input to select either the 'left', 'right' or
 'both' hemispheres.



In [ ]:
# Opaque left hemispehre of the white matter
b_obj_lw = BrainObj('white', hemisphere='left', translucent=False)
sc.add_to_subplot(b_obj_lw, row=0, col=1, rotate='right',
                  title='Left hemisphere', **KW)

Projection iEEG data on the surface of the brain
##############################################################################
 As explain above, we define a source object and project the source's activity
 on the surface of the brain



In [ ]:
xyz.shape,  data.shape

In [ ]:
# First, define a brain object used for the projection
b_obj_proj = BrainObj('B3', hemisphere='both', translucent=False)
# Define the source object
s_obj = SourceObj('iEEG', xyz, data=data, cmap='inferno')
# Just for fun, color sources according to the data :)
s_obj.color_sources(data=data)
# Project source's activity
s_obj.project_sources(b_obj_proj, cmap='plasma')
# Finally, add the source and brain objects to the subplot
sc.add_to_subplot(s_obj, row=0, col=2, title='Project iEEG data', **KW)
sc.add_to_subplot(b_obj_proj, row=0, col=2, rotate='left', use_this_cam=True)
# Finally, add the colorbar :
cb_proj = ColorbarObj(s_obj, cblabel='Projection of niEEG data', **CBAR_STATE)
sc.add_to_subplot(cb_proj, row=0, col=3, width_max=200)

<div class="alert alert-info"><h4>Note</h4><p>Here, we used s_obj.project_sources(b_obj) to project source's activity
    on the surface. We could also have used to b_obj.project_sources(s_obj)</p></div>



Parcellize the brain
##############################################################################
 Here, we parcellize the brain (using all parcellated included in the file).
 Note that those parcellates files comes from MNE-python.



In [ ]:
# Download the annotation file of the left hemisphere lh.aparc.a2009s.annot
path_to_file1 = download_file('lh.aparc.a2009s.annot', astype='example_data')
# Define the brain object (now you should know how to do it)
b_obj_parl = BrainObj('inflated', hemisphere='left', translucent=False)
# Print parcellates included in the file
# print(b_obj_parl.get_parcellates(path_to_file1))
# Finally, parcellize the brain and add the brain to the scene
b_obj_parl.parcellize(path_to_file1)
sc.add_to_subplot(b_obj_parl, row=1, col=1, rotate='left',
                  title='Parcellize using the Desikan Atlas', **KW)

<div class="alert alert-info"><h4>Note</h4><p>Those annotations files from MNE-python are only compatibles with the
    inflated, white and sphere templates</p></div>



Send data to parcellates
##############################################################################
 Again, we download an annotation file, but this time for the right hemisphere
 The difference with the example above, is that this time we send some data
 to some specific parcellates



In [ ]:
# Download the annotation file of the right hemisphere rh.aparc.annot
path_to_file2 = download_file('rh.aparc.annot', astype='example_data')
# Define the brain object (again... I know, this is redundant)
b_obj_parr = BrainObj('inflated', hemisphere='right', translucent=False)
# Print parcellates included in the file
# print(b_obj_parr.get_parcellates(path_to_file2))
# From the list of printed parcellates, we only select a few of them
select_par = ['paracentral', 'precentral', 'fusiform', 'postcentral',
              'superiorparietal', 'superiortemporal', 'inferiorparietal',
              'inferiortemporal']
# Now we define some data for each parcellates (one value per pacellate)
data_par = [10., .1, 5., 7., 11., 8., 4., 6.]
# Parcellize the brain with the selected parcellates. The data range is
# between [.1, 11.]. Then, we use `vmin` and `vmax` to specify that we want
# every parcellates under vmin to be gray and every parcellates over vmax
# darkred
b_obj_parr.parcellize(path_to_file2, select=select_par, hemisphere='right',
                      cmap='viridis', data=data_par, clim=[.1, 11.], vmin=1.,
                      vmax=10, under='gray', over='darkred')
# Add the brain object to the scene
sc.add_to_subplot(b_obj_parr, row=1, col=2, rotate='right',
                  title='Send data to Desikan-Killiany parcellates', **KW)
# Get the colorbar of the brain object and add it to the scene
cb_parr = ColorbarObj(b_obj_parr, cblabel='Data to parcellates', **CBAR_STATE)
sc.add_to_subplot(cb_parr, row=1, col=3, width_max=200)

Custom brain template
##############################################################################
 All of the examples above use MNI brain templates that are included inside
 visbrain. But you can define your own brain template using vertices and faces



In [ ]:
# Download the vertices, faces and normals
mat = np.load(download_file('Custom.npz', astype='example_data'))
vert, faces, norms = mat['vertices'], mat['faces'], mat['normals']
# By default, vertices are in millimeters so we multiply by 1000.
vert *= 1000.
# If your template represent a brain with both hemispheres, you can use the
# `lr_index` to specify which vertices belong to the left or the right
# hemisphere. Basically, `lr_index` is a boolean vector of shape (n_vertices,)
# where True reflect locatino of the left hemisphere and False, the right
# hemisphere
lr_index = vert[0, :] <= 0.
# Create the brain object and add it to the scene (this time it's a bit
# different)
b_obj_custom = BrainObj('Custom', vertices=vert, faces=faces,
                        normals=norms, translucent=False)
sc.add_to_subplot(b_obj_custom, row=2, col=0, title='Use a custom template',
                  rotate='left', **KW)

<div class="alert alert-info"><h4>Note</h4><p>If you doesn't have the normals, it's not a big deal because if no
    normals are provided, normals are going to be computed but it's a bit
    slower. Then, you can save your template using `BrainObj.save`. This can
    be convenient to reload your template later.</p></div>



fMRI activation
##############################################################################
 Add fMRI activations (included in a nii.gz file) to the surface. The provided
 file comes from MNE-python



In [ ]:
# Download the lh.sig.nii.gz file
file = download_file('lh.sig.nii.gz', astype='example_data')
# Define the [...] you know
b_obj_fmri = BrainObj('inflated', translucent=False, sulcus=True)
# Add fMRI activation and hide every activation that is under 5.
b_obj_fmri.add_activation(file=file, clim=(5., 20.), hide_under=5,
                          cmap='viridis', hemisphere='left')
sc.add_to_subplot(b_obj_fmri, row=2, col=1, title='Add fMRI activation',
                  rotate='left', **KW)

MEG inverse solution
##############################################################################
 Finally, plot MEG inverse solution. The provided file comes from MNE-python



In [ ]:
# Dowload meg_source_estimate-rh.stc file and load the data
file = read_stc(download_file('meg_source_estimate-rh.stc',
                              astype='example_data'))
# Get the data of index 2 and the vertices
data = file['data'][:, 2]
vertices = file['vertices']

In [ ]:
data.shape

In [ ]:
# Dowload meg_source_estimate-rh.stc file and load the data
file = read_stc(download_file('meg_source_estimate-rh.stc',
                              astype='example_data'))
# Get the data of index 2 and the vertices
data = file['data'][:, 2]
vertices = file['vertices']
# You know...
b_obj_meg = BrainObj('inflated', translucent=False, hemisphere='right',
                     sulcus=True)
# Add MEG data to the surface and hide every values under 5.
b_obj_meg.add_activation(data=data, vertices=vertices, hemisphere='right',
                         smoothing_steps=21, clim=(5., 17.), hide_under=5.,
                         cmap='plasma')
# Add the brain and the colorbar object to the scene
sc.add_to_subplot(b_obj_meg, row=2, col=2, title='MEG inverse solution',
                  rotate='right', **KW)
cb_parr = ColorbarObj(b_obj_meg, cblabel='MEG data', **CBAR_STATE)
sc.add_to_subplot(cb_parr, row=2, col=3, width_max=200)

"Fun" stuff
##############################################################################
 You can link 3D rotations of subplots which means that if you rotate one
 brain, the other linked object inherit from the same rotations. Finally, you
 can take a screenshot of the scene, without the need to open the window.
 This can be particulary convenient when scenes are included inside loops to
 automatize figure generation.



In [ ]:
%pwd

In [ ]:
CONFIG['MPL_RENDER']

In [ ]:
# Link the rotation of subplots (row=0, col=1) and (row=1, col=2)
# sc.link((0, 1), (1, 2))
# Screenshot of the scene
# sc.screenshot('ex_brain_obj.png', transparent=True)

sc.preview()
#/home/demitau/osccode/data_proc/
#sc.screenshot(saveas='/home/demitau/visbrain.png', print_size=(5, 5))

In [ ]:
#help(fig.savefig)
#    fig.savefig("/home/demitau/osccode/data_proc/output/visbrain.png", dpi=300)


In [ ]:
help(RenderingImShow)

In [ ]:
help(sc)

In [ ]:
import numpy as np
from visbrain.objects import SourceObj  # Import a source object

# Define 100 random 3D (x, y, z) coordinates :
xyz = np.random.rand(100, 3)

# Define a source object :
s_obj = SourceObj('obj_name', xyz, color='green', symbol='square',
                  edge_color='white')

# Object preview with a black background:
s_obj.preview(bgcolor='black')

In [ ]:
import numpy as np
from visbrain.objects import BrainObj, SceneObj, SourceObj

# Define a source and a brain objects :
b_obj_1 = BrainObj('white', translucent=False)
b_obj_2 = BrainObj('B1')
s_obj = SourceObj('my_sources', 50 * np.random.uniform(-1, 1, (100, 3)))

# Define a scene with a black background:
sc = SceneObj(bgcolor='black')

# Add the first brain object to the scene :
sc.add_to_subplot(b_obj_1, row=0, col=0)

# Add the source and the first brain object to same subplot :
sc.add_to_subplot(b_obj_2, row=0, col=1)
sc.add_to_subplot(s_obj, row=0, col=1)

# Finally, display the scene :
sc.preview()